In [34]:
from roboflow import Roboflow
import os

dataset_path = "./taiwan_pill_for_label/"

#download from roboflow
rf = Roboflow(api_key="zCMD7YHOUliH3cklpoLi")
project = rf.workspace("doccampill").project("taiwan_pill_for_label")
version = project.version(35)
dataset = version.download("yolov11", location=dataset_path)


#rename without added roboflow extension
src_path_imgs = dataset_path + 'train/images/'
src_path_labels = dataset_path + 'train/labels/'

for file in os.listdir(src_path_imgs):
    old_name = os.path.basename(file)
    new_name = old_name.split('_')[0]
    os.rename(src_path_imgs + old_name, src_path_imgs + new_name + '.jpg')
    os.rename(src_path_labels + old_name.replace('.jpg', '.txt'), src_path_labels + new_name + '.txt')

#make directories for test and val
os.makedirs(dataset_path + 'val/images', exist_ok=True)
os.makedirs(dataset_path + 'val/labels', exist_ok=True)
os.makedirs(dataset_path + 'test/images', exist_ok=True)
os.makedirs(dataset_path + 'test/labels', exist_ok=True)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ./taiwan_pill_for_label/ in yolov11:: 100%|██████████| 4008/4008 [00:00<00:00, 5760.52it/s]


In [36]:
from sklearn.model_selection import train_test_split
import numpy as np
import os

dataset_path = "./taiwan_pill_for_label/"

x = []
y = []

src_path = dataset_path + 'train/'
for file in os.listdir(src_path + 'labels/'):
    label = []
    with open(src_path + 'labels/' + file, 'r') as f:
        for line in f:
            label.append(line.split(' ')[0])
    f.close()
    if (len(set(label)) > 1):
        print('Error: multiple classes in file')
        print(file)
        break
    else:
        x.append(file)
        y.append(label[0])

x_train, x_temp, y_train, y_temp = train_test_split(x,y,test_size=.2, random_state=42, stratify=y)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=.5, random_state=42)

dest_path = dataset_path + 'val/'
for file in x_val:
    os.rename(src_path + 'labels/' + file, dest_path + 'labels/' + file)
    os.rename(src_path + 'images/' + file.replace('.txt', '.jpg'), dest_path + 'images/' + file.replace('.txt', '.jpg'))

dest_path = dataset_path + 'test/'
for file in x_test:
    os.rename(src_path + 'labels/' + file, dest_path + 'labels/' + file)
    os.rename(src_path + 'images/' + file.replace('.txt', '.jpg'), dest_path + 'images/' + file.replace('.txt', '.jpg'))


In [18]:
from segmentation_mask_overlay import overlay_masks
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

path = './taiwan_pill_for_Label-35/train/'
img_path = path + 'images/-000006-20230922-001_jpg.rf.af2d14c487559c404700ecee58215ce3.jpg'
label_path = path + 'labels/-000006-20230922-001_jpg.rf.af2d14c487559c404700ecee58215ce3.txt'

im = cv2.imread(img_path)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
im = np.array(im)

with open(label_path, 'r') as f:
    labels = list(map(lambda x: x.rstrip().split(' '),f.readlines()))
f.close()


[ WARN:0@2195.917] global loadsave.cpp:241 findDecoder imread_('./taiwan_pill_for_Label-35/train/images/-000006-20230922-001_jpg.rf.af2d14c487559c404700ecee58215ce3.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("richardradli/ogyeiv2")

print("Path to dataset files:", path)

100%|██████████| 3.24G/3.24G [05:37<00:00, 10.3MB/s]

Extracting files...


Path to dataset files: /Users/Amanda/.cache/kagglehub/datasets/richardradli/ogyeiv2/versions/3


In [ ]:
import ultralytics
path = '/Users/Amanda/Desktop/PillRecognition/taiwan_pill_for_label/taiwan_pill_for_label.yaml'

model = ultralytics.YOLO('yolo11n-seg.pt')
results = model.train(data=path, fraction=.1, epochs=10, single_cls=True, plots=True)

Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.2.0 CPU (Apple M4 Max)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\Amanda\Desktop\PillRecognition\taiwan_pill_for_label\taiwan_pill_for_label.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.1, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, 

RuntimeError: Dataset 'C:\Users\Amanda\Desktop\PillRecognition\taiwan_pill_for_label\taiwan_pill_for_label.yaml' error ❌ 'C:\Users\Amanda\Desktop\PillRecognition\taiwan_pill_for_label\taiwan_pill_for_label.yaml' does not exist

In [19]:
import yaml
path = '/Users/Amanda/Desktop/PillRecognition/taiwan_pill_for_label/taiwan_pill_for_label.yaml'
with open(path, 'r') as file:
    print(yaml.safe_load(file))
file.close()

{'path': '/Users/Amanda/Desktop/PillRecognition/taiwan_pill_for_label', 'train': 'train/images', 'val': 'valid/images', 'test': 'test/images', 'nc': 10, 'names': {0: 'capsule', 1: 'pill_doubleround', 2: 'pill_hexagon', 3: 'pill_octagon', 4: 'pill_oval', 5: 'pill_pentagon', 6: 'pill_quadrilateral', 7: 'pill_round', 8: 'pill_triangle', 9: 'pill_waterdrop'}}


In [13]:
import ultralytics

path = '/Users/Amanda/.cache/kagglehub/datasets/richardradli/ogyeiv2'
model = ultralytics.YOLO('yolo11n-seg.pt')
results = model.train(data='/Users/Amanda/Desktop/PillRecognition/ogyeiv2.yaml', fraction=.1, epochs=10, single_cls=True, plots=True, name="ogyeiv2")

Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.2.0 CPU (Apple M4 Max)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/Amanda/Desktop/PillRecognition/ogyeiv2.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=0.1, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ogyeiv2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

train: Scanning /Users/Amanda/.cache/kagglehub/datasets/richardradli/ogyeiv2/versions/3/ogyeiv2/ogyeiv2/train/labels.cache... 314 images, 0 backgrounds, 0 corrupt: 100%|██████████| 314/314 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1672.1±208.6 MB/s, size: 803.5 KB)



val: Scanning /Users/Amanda/.cache/kagglehub/datasets/richardradli/ogyeiv2/versions/3/ogyeiv2/ogyeiv2/valid/labels.cache... 672 images, 0 backgrounds, 0 corrupt: 100%|██████████| 672/672 [00:00<?, ?it/s]


Plotting labels to runs/segment/ogyeiv2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/segment/ogyeiv2
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

       2/10         0G     0.6478     0.5175      1.805     0.8194         10        640: 100%|██████████| 20/20 [04:31<00:00, 13.58s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [01:54<00:00,  5.45s/it]

                   all        672        672    0.00333          1     0.0124     0.0102    0.00333          1     0.0125     0.0102



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       3/10         0G      0.626     0.4842      1.667     0.8438         16        640:  50%|█████     | 10/20 [02:25<02:25, 14.52s/it]


KeyboardInterrupt: 

In [7]:
%pwd

'/Users/Amanda/Desktop/PillRecognition'

In [1]:
from sklearn.model_selection import train_test_split
import os

